In [1]:
# 한국어 금융 뉴스 긍정, 부정 감성 분류
! pip install datasets

                                              0.0/474.6 kB ? eta -:--:--
     ------------------------------------- 474.6/474.6 kB 31.0 MB/s eta 0:00:00
                                              0.0/110.5 kB ? eta -:--:--
     ---------------------------------------- 110.5/110.5 kB ? eta 0:00:00
                                              0.0/132.9 kB ? eta -:--:--
     ---------------------------------------- 132.9/132.9 kB ? eta 0:00:00


In [6]:
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv('finance_data.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   labels        4846 non-null   int64 
 1   sentence      4846 non-null   object
 2   kor_sentence  4846 non-null   object
dtypes: int64(1), object(2)
memory usage: 113.7+ KB


In [9]:
df['labels'].value_counts()
# 긍정, 중립, 부정 >> 3개의 레이블

0    2879
1    1363
2     604
Name: labels, dtype: int64

In [10]:
df

,labels,sentence,kor_sentence
0,0,"According to Gran, the company has no plans to...","Gran에 따르면, 그 회사는 회사가 성장하고 있는 곳이지만, 모든 생산을 러시아로..."
1,0,Technopolis plans to develop in stages an area...,테크노폴리스는 컴퓨터 기술과 통신 분야에서 일하는 회사들을 유치하기 위해 10만 평...
2,2,The international electronic industry company ...,"국제 전자산업 회사인 엘코텍은 탈린 공장에서 수십 명의 직원을 해고했으며, 이전의 ..."
3,1,With the new production plant the company woul...,새로운 생산공장으로 인해 회사는 예상되는 수요 증가를 충족시킬 수 있는 능력을 증가...
4,1,According to the company's updated strategy fo...,"2009-2012년 회사의 업데이트된 전략에 따르면, Basware는 20% - 4..."
...,...,...,...
4841,2,LONDON MarketWatch -- Share prices ended lower...,런던 마켓워치 -- 은행주의 반등이 FTSE 100지수의 약세를 상쇄하지 못하면서 ...
4842,0,Rinkuskiai's beer sales fell by 6.5 per cent t...,린쿠스키아의 맥주 판매량은 416만 리터로 6.5% 감소했으며 카우노 알루스의 맥주...
4843,2,Operating profit fell to EUR 35.4 mn from EUR ...,"영업이익은 2007년 68.8 mn에서 35.4 mn으로 떨어졌으며, 선박 판매 이..."
4844,2,Net sales of the Paper segment decreased to EU...,페이퍼 부문 순매출은 2008년 2분기 241.1 mn에서 2009년 2분기 221...


In [11]:
# 학습을 하기 위해서 [0, 1, 2]로 정수인코딩 해주기

# df['labels'] = df['labels'].replace(['neutral', 'positive', 'negative'],[0,1,2]) > 이미 돼있음

In [12]:
df.to_csv('finance_data.csv', index = False, encoding = 'utf-8-sig')

In [13]:
from datasets import load_dataset

In [14]:
all_data = load_dataset('csv', data_files={'train' : 'finance_data.csv'})
# 모든 데이터가 train에 저장 되어짐

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 1000.55it/s]


Dataset csv downloaded and prepared to C:/Users/SBAUser/.cache/huggingface/datasets/csv/default-5b21fc165b4714fa/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 499.14it/s]


In [15]:
all_data

DatasetDict({
    train: Dataset({
        features: ['labels', 'sentence', 'kor_sentence'],
        num_rows: 4846
    })
})

In [16]:
cs = all_data['train'].train_test_split(0.2) # 8 : 2 train, test data 구분
train_cs = cs['train']
test_cs = cs['test']

In [17]:
print(train_cs)
print()
print(test_cs)

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 3876
})

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 970
})


In [18]:
#  검증 데이터 생성(훈련용 데이터셋에서 분리)
cs = train_cs.train_test_split(0.2)
train_cs = cs['train']
val_cs = cs['test']

In [19]:
print(train_cs)
print()
print(test_cs)

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 3100
})

Dataset({
    features: ['labels', 'sentence', 'kor_sentence'],
    num_rows: 970
})


In [20]:
# 두 번째 샘플 출력
print(train_cs['kor_sentence'][1])

환매는 분배 가능한 자본금과 적립금을 감소시킬 것이다.


In [21]:
# 두 번째 샘플의 레이블 출력
print(train_cs['labels'][1])

2


In [22]:
# 데이터 셋 전처리

import numpy as np
import pandas as pd
import random
import time
import datetime
from tqdm import tqdm

import csv
import os

import tensorflow as tf
import torch

# BERT 사용하기 위한 모듈 부르기
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# 패딩 하기 위한 모듈 부르기
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 전처리 및 평가 지표
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, hamming_loss

In [23]:
# 훈련데이터, 검증데이터, 테스트 데이터에 대해 [CLS] 문장 [SEP] 구조로 만든다
# [CLS] : BERT가 분류 위해 사용하는 첫번째 입력 토큰
# [SEP] : 입력 문장의 종료를 나타내기 위한 스페셜 토큰

train_sequences = list(train_cs['kor_sentence'])
val_sequences = list(val_cs['kor_sentence'])
test_sequences = list(test_cs['kor_sentence'])

In [24]:
train_labels = list(train_cs['labels'])
val_labels = list(val_cs['labels'])
test_labels = list(test_cs['labels'])

In [25]:
train_sequences[:5]

['SWOT 분석은 분류의 한 가지 방법일 뿐이며 그 나름의 단점이 있다.',
 '환매는 분배 가능한 자본금과 적립금을 감소시킬 것이다.',
 '지속가능발전에 대한 의지가 높은 상위 기업이 지수 안에 포함된다.',
 '그러나 시는 공원 벤치, 쓰레기통, 공중화장실, 버스 쉼터, 가로등과 같은 맞춤형 공공가구 공공조달 입찰을 유치할 예정이다.',
 '비경상항목을 제외한 영업이익은 40.6유로로 전년 동기 대비 57.3유로보다 감소했다.']

In [26]:
test_labels[:5] # 0 : 중립, 1: 긍정, 2: 부정

[2, 1, 0, 0, 0]

In [27]:
# BERT TOKENIZER 이용, 처리

tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
MAX_LEN = 128

# 정수 인코딩 과정
# 각 텍스트를 토큰화 >> vocabulary에 mapping 되는 정수 시퀀스로 변환
# [안녕하세요] >> ['안', '녕', '하세요'] >> [231, 52, 45]

def data_to_tensor(sentences, labels, max_len = MAX_LEN) :
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    input_idx = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # pad_sequences
    # 패딩을 위한 모듈
    # >> 주어진 최대 길이를 위해서 뒤에서 0으로 채워줌
    input_idx = pad_sequences(input_idx, maxlen=max_len, dtype = 'long', truncating = 'post', padding = 'post')
    # truncating='post' : maxlen(최대 길이) 초과 부분은 잘라냄
    # padding = 'post' : 0으로 채움
    
    attention_masks = []
    
    for seq in input_idx :
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    # input_idx 요소에서 0이 아닌 부분 >> 1, 
    # 0인 부분 >> 0으로 변환
    
    # 반환된 값, input된 idx, labels를 텐서로 변환
    tensor_inputs = torch.tensor(input_idx)
    tensor_labels = torch.tensor(labels)
    tensor_masks = torch.tensor(attention_masks)
    
    return tensor_inputs, tensor_labels, tensor_masks


In [29]:
# 훈련용 검증용 테스트용 데이터에 대해 
# data_to_tensor 함수 사용 >> 정수 인코딩 된 데이터, 라벨, 어텐션 마스크 얻기

train_inputs, train_labels, train_masks = data_to_tensor(train_sequences, train_labels)
val_inputs, val_labels, val_masks = data_to_tensor(val_sequences, val_labels)
test_inputs, test_labels, test_masks = data_to_tensor(test_sequences, test_labels)

In [30]:
print(train_inputs[0])
print()
print(train_labels[0])
print()
print(train_masks[0])

tensor([ 9815,   429,   421,   426, 10719,  8135, 10585,  8143,  9036,  9340,
         9597,  8149,  9856,  9639,  9022, 47633, 35764, 10960,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0],

In [31]:
# batch_size = 32
# 파이토치 데이터로더(배치 단위로 데이터 꺼내오는 모듈) 변환

batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size = batch_size)


In [32]:
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = RandomSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size = batch_size)

In [33]:
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size = batch_size)

In [34]:
print('훈련 데이터 크기 :', len(train_labels))
print('검증 데이터 크기 :', len(val_labels))
print('테스트 데이터 크기 :', len(test_labels))

훈련 데이터 크기 : 3100
검증 데이터 크기 : 776
테스트 데이터 크기 : 970


In [35]:
# GPU 정상 세팅 여부 확인

if torch.cuda.is_available() :
    device = torch.device('cuda')
    print(torch.cuda.device_count()) # 몇 개 gpu 사용 가능 확인
    print(torch.cuda.get_device_name(0))
else :
    device = torch.device('cpu')

1
NVIDIA GeForce RTX 3070


In [3]:
# 모델 로드하기 

# 텍스트 분류 BERT : BertForSequenceClassfication.from_pretrained('모델 이름')

num_labels = 3

model = AutoModelForSequenceClassification.from_pretrained('skt/kogpt2-base-v2', num_labels = num_labels)
model.cuda()

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=3, bias=False)
)

In [53]:
# optimizer 선택

optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)

c:\Users\SBAUser\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [56]:
epochs = 10

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [37]:
def format_time(elapsed) :
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds = elapsed_rounded)) # hh : mm : ss

In [38]:
def metrics(predictions, labels) :
    y_pred = predictions
    y_true = labels
    
    # 사용 가능한 metrics 사용
    
    acc = accuracy_score(y_true, y_pred)
    f1_macro_avg = f1_score(y_true = y_true, y_pred=y_pred, average='macro', zero_division=0)
    f1_micro_avg = f1_score(y_true = y_true, y_pred=y_pred, average='micro', zero_division=0)
    f1_weighted_avg = f1_score(y_true = y_true, y_pred=y_pred, average='weighted', zero_division=0)
    
    result = {'accuracy' : acc, 'f1_macro_average' : f1_macro_avg, 'f1_micro_average' : f1_micro_avg, 'f1_weighted_average' : f1_weighted_avg}
    
    return result

In [59]:
# 모델 학습

seed_val = 777
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()
for epoch in range(epochs) :
    print('----- Epoch {:} / {:} -----'.format(epoch+1, epochs))
    t0 = time.time() # 현재 시간 >> t0 저장
    total_loss = 0 
    
    model.train()
    
    for step,batch in tqdm(enumerate(train_dataloader), total = len(train_dataloader)) :
        if step % 500 == 0 and not step ==0 :
            elapsed = format_time(time.time() - t0)
            print('Batch {:>5, } of {:5,}. Elapsed : {:}.'.format(step, len(train_dataloader), elapsed))
            
        batch = tuple(t.to(device) for t in batch)
        b_input_idx, b_input_mask, b_labels = batch
        # 각 배치를 device에 할당, b_input_idx, b_input_mask, b_labels 데이터 저장
        
        outputs = model(b_input_idx, attention_mask = b_input_mask, labels = b_labels)
        
        loss = outputs[0] # output의 첫번째 요소 >> loss
        total_loss += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)
        # clip_grad_norm
        optimizer.step()
        scheduler.step()
        
        model.zero_grad()
    
    avg_train_loss = total_loss / len(train_dataloader)
    
    print()
    print('Average training loss : {0:.4f}'.format(avg_train_loss))
    print('Training epoch took : {:}'.format(format_time(time.time()-t0)))
    

----- Epoch 1 / 10 -----


  0%|          | 0/97 [00:00<?, ?it/s]C:\Users\SBAUser\AppData\Local\Temp\ipykernel_17844\287155217.py:31: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)
100%|██████████| 97/97 [00:36<00:00,  2.63it/s]



Average training loss : 0.5707
Training epoch took : 0:00:37
----- Epoch 2 / 10 -----


100%|██████████| 97/97 [00:34<00:00,  2.83it/s]



Average training loss : 0.3066
Training epoch took : 0:00:34
----- Epoch 3 / 10 -----


...


Average training loss : 0.0064
Training epoch took : 0:00:33
----- Epoch 9 / 10 -----


100%|██████████| 97/97 [00:33<00:00,  2.92it/s]



Average training loss : 0.0014
Training epoch took : 0:00:33
----- Epoch 10 / 10 -----


100%|██████████| 97/97 [00:33<00:00,  2.91it/s]


Average training loss : 0.0001
Training epoch took : 0:00:33


In [67]:
# 검증 데이터에 대한 평가

t0 = time.time()
model.eval()
# 모델 평가 모드 전환
accum_logits, accum_label_idx = [], []

for batch in val_dataloader :
    batch = tuple(t.to(device) for t in batch)
    b_input_idx, b_input_mask, b_labels = batch
    
    with torch.no_grad() :
        outpus = model(b_input_idx, token_type_ids = None, attention_mask = b_input_mask)
        # token_type_ids = None : 하나의 문장으로만 이루어진 경우에 설정
        # BERT 모델 사용 시, 입력 데이터가 2개 문장으로 이루어진 경우 사용
        # >> 두 문장 사이 경계 구분할 때 사용
        # token_type_ids = None 경계가 없기 때문에 None으로 설정
        
        
        logits = outpus[0]
        logits = logits.detach().cpu().numpy()
        label_idx = b_labels.to('cpu').numpy()
        
        for b in logits :
            accum_logits.append(np.argmax(b))
            
        for b in label_idx :
            accum_label_idx.append(b)
    
accum_logits = np.array(accum_logits)
accum_label_idx = np.array(accum_label_idx)
result = metrics(accum_logits, accum_label_idx)

        

In [68]:
print('Accuracy : {0:.4f}'.format(result['accuracy']))
print('f1_macro_score : {0:.4f}'.format(result['f1_macro_average']))
print('f1_micro_score : {0:.4f}'.format(result['f1_micro_average']))
print('f1_weighted_score : {0:.4f}'.format(result['f1_weighted_average']))

Accuracy : 0.8492
f1_macro_score : 0.8403
f1_micro_score : 0.8492
f1_weighted_score : 0.8496


In [69]:
# 모델 저장과 로드

torch.save(model.state_dict(), './GPT_news_sentiment_analysis_model.pt')


In [4]:
model.load_state_dict(torch.load('./GPT_news_sentiment_analysis_model.pt'))


<All keys matched successfully>

In [39]:
# 검증 데이터에 대한 평가

t0 = time.time()
model.eval()
# 모델 평가 모드 전환
accum_logits, accum_label_idx = [], []

for step, batch in tqdm(enumerate(test_dataloader)) :
    if step % 100 == 0 and not step == 0 :
        elapsed = format_time(time.time() - t0)
        print('Batch {:>5,} of {:>,}. Elapsed : {:}'.format(step, len(test_dataloader), elapsed))
    batch = tuple(t.to(device) for t in batch)
    b_input_idx, b_input_mask, b_labels = batch
    
    with torch.no_grad() :
        outpus = model(b_input_idx, token_type_ids = None, attention_mask = b_input_mask)
        # token_type_ids = None : 하나의 문장으로만 이루어진 경우에 설정
        # BERT 모델 사용 시, 입력 데이터가 2개 문장으로 이루어진 경우 사용
        # >> 두 문장 사이 경계 구분할 때 사용
        # token_type_ids = None 경계가 없기 때문에 None으로 설정
        
        
    logits = outpus[0]
    logits = logits.detach().cpu().numpy()
    label_idx = b_labels.to('cpu').numpy()
        
    for b in logits :
        accum_logits.append(np.argmax(b))
            
    for b in label_idx :
        accum_label_idx.append(b)
    
accum_logits = np.array(accum_logits)
accum_label_idx = np.array(accum_label_idx)
result = metrics(accum_logits, accum_label_idx)


31it [00:03,  8.81it/s]


In [40]:
result

{'accuracy': 0.9597938144329897,
 'f1_macro_average': 0.9532493112947659,
 'f1_micro_average': 0.9597938144329897,
 'f1_weighted_average': 0.9599476697622902}

In [49]:
# 예측

from transformers import pipeline

pipe = pipeline('text-classification', model=model.cuda(), tokenizer = tokenizer, device = 0, max_length=512, return_all_scores = False, function_to_apply='softmax')


c:\Users\SBAUser\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [50]:
result = pipe('주식회사 한시경 매출이 급성장했다네요')
result

[{'label': 'LABEL_1', 'score': 0.9999961853027344}]

In [51]:
label_dict = {'LABEL_0' : '중립', 'LABEL_1' : '긍정', 'LABEL_2' : '부정'}

In [52]:
def prediction(text) :
    result = pipe(text)
    
    return label_dict[result[0]['label']]

In [55]:
prediction('서울시 세싹 강동캠퍼스 교육생 전원 취업했대')


'긍정'

In [56]:
prediction('배가 불러서 너무 졸려요')

'부정'

In [57]:
prediction('아버지가 방에 들어가셨다')

'중립'